In [1]:
import csv
import pandas
import string
import warnings
import os
import re
from datetime import datetime
from pprint import pprint
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
! pip install openpyxl==3.0.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 242 kB 26.1 MB/s 
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.0.10
    Uninstalling openpyxl-3.0.10:
      Successfully uninstalled openpyxl-3.0.10


In [5]:
data = pandas.read_excel('drive/MyDrive/Exam2020_Task_7_Essays_mistakes_table.xlsx')

In [ ]:
sentences = pandas.read_excel('drive/MyDrive/Exam2020_Task_7_Essays_tokenized_text_table.xlsx')

In [ ]:
punctuation = data.loc[data['mistake_type'] == 'punct']

In [ ]:
punctuation.loc[(punctuation.correction == 'none'), 'correction'] = ''

In [ ]:
def get_text(text_id, sentence_id):
    text = sentences[sentences['text_id'] == text_id]
    text = text.loc[text['sentence_id'].isin([sentence_id, sentence_id+1])]
    
    words = list(text.token)
    ids = list(text.token_id)
    d = {}

    for el in range(len(words)):
      if isinstance(words[el],datetime):
        return [text_id, sentence_id]
      else:
        d[ids[el]] = str(words[el])

    words = [str(i) for i in words]
    text  = ' '.join(words)
    for k in string.punctuation: 
      if k != '(':
        text = text.replace(f' {k}',k)
      elif k == '(':
        text = text.replace(f'{k} ',k)

    return text, d

In [ ]:
full_texts = []
tokens_ids = []
for el in range(len(punctuation.text_id)):
  full_texts.append(get_text(list(punctuation.text_id)[el], list(punctuation.sentence_id)[el])[0])
  tokens_ids.append(get_text(list(punctuation.text_id)[el], list(punctuation.sentence_id)[el])[1])

In [ ]:
punctuation['full_text'] = full_texts

In [ ]:
punctuation['tokens_ids'] = tokens_ids

In [ ]:
strange_cases = []
for el in punctuation.values:
  if el[0] == el[6]:
    if el[1] == el[7]:
      strange_cases.append(el)

In [ ]:
strange_cases = pandas.DataFrame(strange_cases)
strange_cases.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids'}, inplace = True)

In [ ]:
punctuation = punctuation[punctuation['full_text'].apply(lambda x: x not in strange_cases['full_text'].values)]

In [ ]:
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER, CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

def custom_tokenizer(nlp):
    infixes = (
        LIST_ELLIPSES
        + LIST_ICONS
        + [
            r"(?<=[0-9])[+\-\*^](?=[0-9-])",
            r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
                al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
            ),
            r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
            #r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
            r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
        ]
    )

    infix_re = compile_infix_regex(infixes)

    return Tokenizer(nlp.vocab, prefix_search=nlp.tokenizer.prefix_search,
                                suffix_search=nlp.tokenizer.suffix_search,
                                infix_finditer=infix_re.finditer,
                                token_match=nlp.tokenizer.token_match,
                                rules=nlp.Defaults.tokenizer_exceptions)


nlp = spacy.load("en")
nlp.tokenizer = custom_tokenizer(nlp)

In [ ]:
dependencies = []
for el in list(punctuation['tokens_ids']):
  el = ' '.join(list(el.values()))
  doc = nlp(el)
  d = []
  for token in doc:
    d.append([token.text, token.pos_, token.tag_, token.dep_])
  dependencies.append(d)

In [ ]:
  norm_values = []
  norm_deps = []
  strange_values = []
  strange_deps = []
  for i in range(len(dependencies)):
    if len(dependencies[i]) == len(punctuation.values[i][-1].values()):
      norm_values.append(punctuation.values[i])
      norm_deps.append(dependencies[i])
    else:
      strange_values.append(punctuation.values[i])
      strange_deps.append(dependencies[i])

In [ ]:
def get_new_dependencies(dependencies, punct):
    new_depends = dependencies
    for k in range(len(dependencies)):
      new_d = []
      if dependencies[k][0].isnumeric():
        if dependencies[k+1][0] == '-':
          if dependencies[k+2][0].isnumeric:
            word = ''.join([dependencies[k][0],dependencies[k+1][0],dependencies[k+2][0]])
            poses = ' '.join([dependencies[k][1],dependencies[k+1][1],dependencies[k+2][1]])
            tags = ' '.join([dependencies[k][2],dependencies[k+1][2],dependencies[k+2][2]])
            deps = ' '.join([dependencies[k][3],dependencies[k+1][3],dependencies[k+2][3]])
            new_d.append(word)
            new_d.append(poses)
            new_d.append(tags)
            new_d.append(deps)
            index = k
            break
          
      
            new_depends.insert(index, new_d)
            new_depends.pop(index+1)
            new_depends.pop(index+1)
            new_depends.pop(index+1)
      elif dependencies[i][k][0] == 'I':
        if dependencies[i][k+1][0] =='"':
          if dependencies[i][k+1][0] =='m':
            word = ''.join([dependencies[i][k][0],dependencies[i][k+1][0]])
            poses = ' '.join([dependencies[i][k][1],dependencies[i][k+1][1]])
            tags = ' '.join([dependencies[i][k][2],dependencies[i][k+1][2]])
            deps = ' '.join([dependencies[i][k][3],dependencies[i][k+1][3]])
            new_d.append(word)
            new_d.append(poses)
            new_d.append(tags)
            new_d.append(deps)
            index = k
            break
            new_depends.insert(index, new_d)
            new_depends.pop(index+1)
            new_depends.pop(index+1)

      elif dependencies[k][0] == 'should':
        if dependencies[k+1][0] =='nt':
          word = ''.join([dependencies[k][0],dependencies[i][k+1][0]])
          poses = ' '.join([dependencies[i][k][1],dependencies[i][k+1][1]])
          tags = ' '.join([dependencies[i][k][2],dependencies[i][k+1][2]])
          deps = ' '.join([dependencies[i][k][3],dependencies[i][k+1][3]])
          new_d.append(word)
          new_d.append(poses)
          new_d.append(tags)
          new_d.append(deps)
          index = k
          break
          new_depends.insert(index, new_d)
          new_depends.pop(index+1)
          new_depends.pop(index+1)

      elif dependencies[k][0] == 'That':
        if dependencies[k+1][0] =='s':
          word = ''.join([dependencies[i][k][0],dependencies[i][k+1][0]])
          poses = ' '.join([dependencies[i][k][1],dependencies[i][k+1][1]])
          tags = ' '.join([dependencies[i][k][2],dependencies[i][k+1][2]])
          deps = ' '.join([dependencies[i][k][3],dependencies[i][k+1][3]])
          new_d.append(word)
          new_d.append(poses)
          new_d.append(tags)
          new_d.append(deps)
          index = k
          break
          new_depends.insert(index, new_d)
          new_depends.pop(index+1)
          new_depends.pop(index+1)

      elif dependencies[i][k][0] == 'that':
        if dependencies[i][k+1][0] == 's':
          word = ''.join([dependencies[i][k][0],dependencies[i][k+1][0]])
          poses = ' '.join([dependencies[i][k][1],dependencies[i][k+1][1]])
          tags = ' '.join([dependencies[i][k][2],dependencies[i][k+1][2]])
          deps = ' '.join([dependencies[i][k][3],dependencies[i][k+1][3]])
          new_d.append(word)
          new_d.append(poses)
          new_d.append(tags)
          new_d.append(deps)
          index = k
          break
          new_depends.insert(index, new_d)
          new_depends.pop(index+1)
          new_depends.pop(index+1)

      elif dependencies[i][k][0] == 'can':
        if dependencies[i][k+1][0] == 't':
          word = ''.join([dependencies[i][k][0],dependencies[i][k+1][0]])
          poses = ' '.join([dependencies[i][k][1],dependencies[i][k+1][1]])
          tags = ' '.join([dependencies[i][k][2],dependencies[i][k+1][2]])
          deps = ' '.join([dependencies[i][k][3],dependencies[i][k+1][3]])
          new_d.append(word)
          new_d.append(poses)
          new_d.append(tags)
          new_d.append(deps)
          index = k
          break
          new_depends.insert(index, new_d)
          new_depends.pop(index+1)
          new_depends.pop(index+1)

In [ ]:
norm_values = pandas.DataFrame(norm_values)
norm_values.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids'}, inplace = True)

In [ ]:
dicts_of_deps = []
for i, b in enumerate(norm_values.values):
  d_v = {}
  for k in range(len(list(b[7].keys()))):
    d_v[list(b[7].keys())[k]] = norm_deps[i][k]
  dicts_of_deps.append(d_v)

In [ ]:
norm_values['dependencies'] = dicts_of_deps

In [ ]:
norm_values

In [ ]:
def get_clean_text(sample):
  id = int(sample[2])
  change = sample[4]
  if isinstance(change, float):
    change = str(int(change))
  else:
    change = change
  rechange = sample[5]
  change = re.findall(r"[\w']+|[.,!?;()%]", str(change))
  rechange2 = re.findall(r"[\w']+|[.,!?;()%]", str(rechange))

  change_len = len(change)
  text_with_ids = sample[7]
  end = id + change_len 
  
  all_words = []
  for el in list(text_with_ids.keys()):
    try:
      if el not in [i for i in range(id,end)]:
        all_words.append(str(text_with_ids[el]))
      elif all_words[-1] != rechange:
        all_words.append(str(rechange))

    except IndexError:
      if len(all_words) == 0:
        all_words.append(str(rechange))
  final_text = ' '.join(all_words)
  for k in string.punctuation: 
      if k != '(':
        final_text = final_text.replace(f' {k}',k)
      elif k == '(':
        final_text = final_text.replace(f'{k} ',k)

  final_text2 = re.findall(r"[\w']+|[.,!?;()%]", str(final_text))
  dict_words_clean = {}
  ids_clean = [i for i in range(list((text_with_ids.keys()))[0], len(final_text2) + list(text_with_ids.keys())[0])]

  for k in range(len(ids_clean)):
    dict_words_clean[ids_clean[k]] = final_text2[k]

  return final_text, dict_words_clean

In [25]:
corrected_text = []
corrected_with_ids = []
for element in norm_values.values:
  corrected_text.append(get_clean_text(element)[0])
  corrected_with_ids.append(get_clean_text(element)[1])

In [26]:
norm_values['corrected_text'] = corrected_text

In [27]:
norm_values['corrected_with_ids'] = corrected_with_ids

In [28]:
dependencies_heptabot = []

for el in norm_values.values:
  el = ' '.join(list(el[10].values()))

  doc = nlp(el)
  d = []
  for token in doc:
    d.append([token.text, token.pos_, token.tag_, token.dep_])
  dependencies_heptabot.append(d)

In [29]:
dicts_of_deps_heptabot = []
for i, b in enumerate(norm_values.values):
  d_v = {}
  for k in range(len(list(b[10].keys()))):
    d_v[list(b[10].keys())[k]] = dependencies_heptabot[i][k]
  dicts_of_deps_heptabot.append(d_v)

In [30]:
norm_values['dependencies_heptabot'] = dicts_of_deps_heptabot

In [31]:
punctuation_all = punctuation

In [32]:
punctuation = norm_values

In [33]:
punctuation.loc[(punctuation.correction == 'none'), 'correction'] = ''

## Articles:

In [34]:
bool_series = punctuation["correction"].str.lower().str.endswith('the', na = False)
  
articles_mistakes = punctuation[bool_series]

In [35]:
articles_a_the = []
for el in articles_mistakes.values:
  if 'an' in str(el[4]).split():
    if 'the' in str(el[5]).split():
      articles_a_the.append(el)
  elif 'a' in str(el[4]).split():
    if 'the' in str(el[5]).split():
      articles_a_the.append(el)
  elif 'the' in str(el[4]).split():
    if 'a' in str(el[5]).split():
      articles_a_the.append(el)
  elif 'the' in str(el[4]).split():
    if 'an' in str(el[5]).split():
      articles_a_the.append(el)

In [36]:
articles_a_the = pandas.DataFrame(articles_a_the)
articles_a_the.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
articles_a_the = articles_a_the.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [37]:
articles_a_the

,text_id,sentence_id,token_id,mistake_type,error_span,correction,full_text,tokens_ids,dependencies,corrected_text,corrected_with_ids,dependencies_heptabot
0,440.0,10.0,213.0,punct,a,", the",As an example a quick increase of literacy in ...,"{210: 'As', 211: 'an', 212: 'example', 213: 'a...","{210: ['As', 'SCONJ', 'IN', 'prep'], 211: ['an...","As an example, the quick increase of literacy ...","{210: 'As', 211: 'an', 212: 'example', 213: ',...","{210: ['As', 'SCONJ', 'IN', 'prep'], 211: ['an..."
1,701.0,10.0,172.0,punct,an,", the","From the other hand, if talk about university ...","{162: 'From', 163: 'the', 164: 'other', 165: '...","{162: ['From', 'ADP', 'IN', 'prep'], 163: ['th...","From the other hand, if talk about university ...","{162: 'From', 163: 'the', 164: 'other', 165: '...","{162: ['From', 'ADP', 'IN', 'prep'], 163: ['th..."


In [38]:
def change_articles_a_the(sample):
  tag = 'punctuation'
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))


  i = list(sample[7].keys())[0]
  index1 = list(sample[7].keys())[0]
  while i != sample[2]:
    i += 1
    if sample[7][i] == ',':
      index1 = i + 1
  index2 = int(sample[2]) 



  indexes = range(index1, index2)
  error_area = []
  for b in indexes:
    error_area.append(str(sample[7][b]))

  

  pprint('CORRECTION: ' + ' '.join(error_area) + ',')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

  tag = 'articles'
  article = re.findall(r'[a-zA-Z]+', str(sample[5]))[0]
  correction = [article, str(sample[7][int(sample[2])+1])]
  error_area = [sample[7][int(sample[2])], sample[7][int(sample[2])+1]]
  pprint('CORRECTION: ' + ' '.join(correction))
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

In [39]:
#for el in articles_a_the.values:
#  change_articles_a_the(el)

In [40]:
articles_mistakes = articles_mistakes[articles_mistakes['corrected_text'].apply(lambda x: x not in articles_a_the['corrected_text'].values)]

In [41]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in articles_a_the['corrected_text'].values)]

In [42]:
articles_without_punct = []
for el in articles_mistakes.values:
  for i in string.punctuation:
    if i not in str(el[4]):
      if i in str(el[5]):
        articles_without_punct.append(el)

In [43]:
articles_without_punct = pandas.DataFrame(articles_without_punct)
articles_without_punct.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
articles_without_punct = articles_without_punct.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [44]:
def articles_correction_commas(sample):

  tag = 'punctuation'
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  i = list(sample[7].keys())[0]
  index1 = list(sample[7].keys())[0]
  while i != sample[2]:
    i += 1
    if sample[7][i] == ',':
      index1 = i + 1

  index2 = int(sample[2]) + 1
  indexes = range(index1, index2)

  error_area = []
  for b in indexes:
    error_area.append(str(sample[7][b]))

  pprint('CORRECTION: ' + ' '.join(error_area) + ',')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)


  error_area2 = []
  tag = 'articles'
  indexes = []
  ind = int(sample[2]+1)
  while not (sample[8][ind][1] == 'NOUN' or sample[8][ind][1] == 'PROPN'):
    indexes.append(ind)
    ind += 1
  indexes.append(ind)
      

  for b in indexes:
    error_area2.append(str(sample[7][b]))
  pprint('ERROR AREA: '+ ' '.join(error_area2))
  pprint('CORRECTION: ' + 'the ' + ' '.join(error_area2))
  pprint('TAG: ' + tag) 
  print('\n')

In [45]:
#for el in articles_without_punct.values:
#  articles_correction_commas(el)
  

In [46]:
articles_mistakes = articles_mistakes[articles_mistakes['corrected_text'].apply(lambda x: x not in articles_without_punct['corrected_text'].values)]

In [47]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in articles_without_punct['corrected_text'].values)]

In [48]:
articles_with_the = []
for el in articles_mistakes.values:
  if 'and' in str(el[5]):
    if 'the' in str(el[5]):
      articles_with_the.append(el)

In [49]:
articles_with_the = pandas.DataFrame(articles_with_the)
articles_with_the.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
articles_with_the = articles_with_the.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [50]:
def articles_correction_dots(sample):
  element = str(el[4])
  f = re.findall(r'[a-zA-Z]+', element)
  a = re.findall(r'\d+', element)
  k = re.findall(r'\W', element)
  all = f+a+k
  if ' ' in all:
    all.remove(' ')

  length = len(all)

  indexes = []
  ind = int(sample[2]+length)
  while not (sample[8][ind][1] == 'NOUN' or sample[8][ind][1] == 'PROPN'):
    indexes.append(ind)
    ind += 1
  indexes.append(ind)

  error_area = []
  for i in indexes:
    error_area.append(sample[7][i])

  tag = 'articles'
  pprint('CORRECTION: ' + 'the ' + ' '.join(error_area))
  pprint('ERROR AREA: '+ str(sample[7][int(sample[2])+length]))
  pprint('TAG: ' + tag)

  tag = 'Absence of a component in clause or sentence'
  pprint('CORRECTION: ' + str(sample[7][int(sample[2])]) + ', and')
  pprint('ERROR AREA: '+ str(sample[7][int(sample[2])]))
  pprint('TAG: ' + tag)



In [51]:
#for el in articles_with_the.values:
#  articles_correction_dots(el)

In [52]:
articles_mistakes = articles_mistakes[articles_mistakes['corrected_text'].apply(lambda x: x not in articles_with_the['corrected_text'].values)]

In [53]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in articles_with_the['corrected_text'].values)]

## Commas are changed with dots:

In [54]:
bool_series = punctuation["error_span"].str.lower().str.endswith(',', na = False)
commas_dot = punctuation[bool_series]
bool_series = commas_dot['correction'].str.endswith('.', na = False)
commas_dot = commas_dot[bool_series]

In [55]:
# это первое
def checking_commas_dots(sample):
  tag = 'punctuation'
  parts_of_sentences = ['ROOT', 'relcl', 'nsubj', 'expl', 'conj', 'ccomp', 'aux', 
                      'neg', 'nsubjpass','punct','intj', 'acl']

  all_sents = []
  first_sent = []
  for el in sample[11].values():
    if el[-1] in parts_of_sentences:
      all_sents.append(el)

  for i in all_sents:
    first_sent.append(i)
    if i[0] == '.':
      ind = all_sents.index(i)
      break
      
  second_sent = all_sents[ind::]
  del first_sent[-1]

  if len(first_sent) >= 4:
    pass

  else:
    parts_of_sentences2 = ['nsubj','ROOT', 'expl']
    sent = []
    for el in first_sent:
      if el in parts_of_sentences2:
        sent.append(el)
        if len(sent) >= 2:
          pass

  index1 = int(list(sample[8].keys())[0])
  index2 = int(sample[2]) + 1
  indexes = range(index1, index2)
  error_area = [sample[7][i] for i in indexes]

  print("USER'S TEXT: " + sample[6])
  print('CORRECTED TEXT: ' + str(sample[9]))

  pprint('CORRECTION: ' + ' '.join(error_area[0:-1]) + '.')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

    

In [56]:
#for el in commas_dot.values:
#  checking_commas_dots(el)

In [57]:
punctuation = punctuation[~punctuation.isin(commas_dot)].dropna()

In [58]:
numeral_procents = []
for el in punctuation.values:
  if str(el[4]).isnumeric():
    if str(el[5]).endswith('%'):
      numeral_procents.append(el)

In [59]:
numeral_procents = pandas.DataFrame(numeral_procents)
numeral_procents.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
numeral_procents = numeral_procents.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [60]:
def num_procents(sample): 
  text = str(sample[5])
  find = re.findall(r"[\w']+|[.,!?;()%]", text)


  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))

  pprint('CORRECTION: ' + ''.join(find))
  pprint('ERROR AREA: '+ str(sample[7][sample[2]]))
  pprint('TAG: numerals')

In [61]:
#for el in numeral_procents.values:
#  num_procents(el)

In [62]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in numeral_procents['corrected_text'].values)]

In [63]:
commas_delete = punctuation[punctuation.correction == '']
commas_delete = commas_delete[commas_delete.error_span == ',']
commas_delete

,text_id,sentence_id,token_id,mistake_type,error_span,correction,full_text,tokens_ids,dependencies,corrected_text,corrected_with_ids,dependencies_heptabot
172,14.0,2.0,22.0,punct,",",,Almost from start we already have some life's ...,"{12: 'Almost', 13: 'from', 14: 'start', 15: 'w...","{12: ['Almost', 'ADV', 'RB', 'advmod'], 13: ['...",Almost from start we already have some life's ...,"{12: 'Almost', 13: 'from', 14: 'start', 15: 'w...","{12: ['Almost', 'ADV', 'RB', 'advmod'], 13: ['..."
173,20.0,1.0,16.0,punct,",",,As many people values benefits and opportuniti...,"{1: 'As', 2: 'many', 3: 'people', 4: 'values',...","{1: ['As', 'SCONJ', 'IN', 'mark'], 2: ['many',...",As many people values benefits and opportuniti...,"{1: 'As', 2: 'many', 3: 'people', 4: 'values',...","{1: ['As', 'SCONJ', 'IN', 'mark'], 2: ['many',..."
174,44.0,2.0,44.0,punct,",",,"However, other think that education is a speci...","{30: 'However', 31: ',', 32: 'other', 33: 'thi...","{30: ['However', 'ADV', 'RB', 'advmod'], 31: [...","However, other think that education is a speci...","{30: 'However', 31: ',', 32: 'other', 33: 'thi...","{30: ['However', 'ADV', 'RB', 'advmod'], 31: [..."
175,44.0,14.0,270.0,punct,",",,"To conclude, education is an important both fo...","{260: 'To', 261: 'conclude', 262: ',', 263: 'e...","{260: ['To', 'PART', 'TO', 'aux'], 261: ['conc...","To conclude, education is an important both fo...","{260: 'To', 261: 'conclude', 262: ',', 263: 'e...","{260: ['To', 'PART', 'TO', 'aux'], 261: ['conc..."
176,48.0,8.0,158.0,punct,",",,"Another point of view, that I support, is that...","{154: 'Another', 155: 'point', 156: 'of', 157:...","{154: ['Another', 'DET', 'DT', 'det'], 155: ['...","Another point of view that I support, is that...","{154: 'Another', 155: 'point', 156: 'of', 157:...","{154: ['Another', 'DET', 'DT', 'det'], 155: ['..."
...,...,...,...,...,...,...,...,...,...,...,...,...
845,1696.0,9.0,205.0,punct,",",,"However, there are people, who think that stud...","{200: 'However', 201: ',', 202: 'there', 203: ...","{200: ['However', 'ADV', 'RB', 'advmod'], 201:...","However, there are people who think that stud...","{200: 'However', 201: ',', 202: 'there', 203: ...","{200: ['However', 'ADV', 'RB', 'advmod'], 201:..."
846,1697.0,3.0,55.0,punct,",",,"It is also clear, that the percentage differen...","{51: 'It', 52: 'is', 53: 'also', 54: 'clear', ...","{51: ['It', 'PRON', 'PRP', 'nsubj'], 52: ['is'...",It is also clear that the percentage differen...,"{51: 'It', 52: 'is', 53: 'also', 54: 'clear', ...","{51: ['It', 'PRON', 'PRP', 'nsubj'], 52: ['is'..."
847,1697.0,6.0,126.0,punct,",",,"It is significant, that the percentage differe...","{123: 'It', 124: 'is', 125: 'significant', 126...","{123: ['It', 'PRON', 'PRP', 'nsubj'], 124: ['i...",It is significant that the percentage differe...,"{123: 'It', 124: 'is', 125: 'significant', 126...","{123: ['It', 'PRON', 'PRP', 'nsubj'], 124: ['i..."
848,1698.0,4.0,79.0,punct,",",,"First of all, it seems to me, that the ability...","{71: 'First', 72: 'of', 73: 'all', 74: ',', 75...","{71: ['First', 'ADV', 'RB', 'advmod'], 72: ['o...","First of all, it seems to me that the ability...","{71: 'First', 72: 'of', 73: 'all', 74: ',', 75...","{71: ['First', 'ADV', 'RB', 'advmod'], 72: ['o..."


In [64]:
that_commas_delete = []
for el in commas_delete.values:
  if str(el[7][int(el[2])+1]) == 'that':
    that_commas_delete.append(el)

In [65]:
that_commas_delete = pandas.DataFrame(that_commas_delete)
that_commas_delete.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
that_commas_delete = that_commas_delete.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [66]:
def change_relative_comma(sample):
  tag = 'relative clause'

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))

  error_area = [str(el[7][int(el[2])]), str(el[7][int(el[2])+1])]
  pprint('ERROR AREA: ' + ' '.join(error_area))
  pprint('CORRECTION: '+ str(sample[7][int(sample[2]+1)]))
  pprint('TAG: ' + tag)

In [67]:
#for el in that_commas_delete.values:
#  change_relative_comma(el)

In [68]:
commas_delete = commas_delete[commas_delete['corrected_text'].apply(lambda x: x not in that_commas_delete['corrected_text'].values)]

In [69]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in that_commas_delete['corrected_text'].values)]

In [70]:
whos_whichs_commas_delete = []
for el in commas_delete.values:
  if str(el[7][int(el[2])+1]) == 'who':
    whos_whichs_commas_delete.append(el)
  if str(el[7][int(el[2])+1]) == 'which':
    whos_whichs_commas_delete.append(el)

In [71]:
whos_whichs_commas_delete = pandas.DataFrame(whos_whichs_commas_delete)
whos_whichs_commas_delete.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
whos_whichs_commas_delete = whos_whichs_commas_delete.drop_duplicates(subset=['corrected_text'], ignore_index=True)

Уточнить завтра

In [72]:
#for el in whos_whichs_commas_delete.values:
#  change_relative_comma(el)

In [73]:
commas_delete = commas_delete[commas_delete['corrected_text'].apply(lambda x: x not in whos_whichs_commas_delete['corrected_text'].values)]

In [74]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in whos_whichs_commas_delete['corrected_text'].values)]

In [75]:
thans_commas_delete = []
for el in commas_delete.values:
  if str(el[7][int(el[2])+1]) == 'than':
    thans_commas_delete.append(el)

In [76]:
thans_commas_delete = pandas.DataFrame(thans_commas_delete)
thans_commas_delete.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
thans_commas_delete = thans_commas_delete.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [77]:
#for el in thans_commas_delete.values:
#  change_relative_comma(el)

In [78]:
commas_delete = commas_delete[commas_delete['corrected_text'].apply(lambda x: x not in thans_commas_delete['corrected_text'].values)]

In [79]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in thans_commas_delete['corrected_text'].values)]

In [80]:
conjunctions = []
for el in commas_delete.values:
  if el[8][int(el[2])+1][3] == 'advmod':
    conjunctions.append(el)

In [81]:
conjunctions = pandas.DataFrame(conjunctions)
conjunctions.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
conjunctions = conjunctions.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [82]:
#for el in conjunctions.values:
#  change_relative_comma(el)

In [83]:
commas_delete = commas_delete[commas_delete['corrected_text'].apply(lambda x: x not in conjunctions['corrected_text'].values)]

In [84]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in conjunctions['corrected_text'].values)]

In [85]:
sconjs = []
for el in commas_delete.values:
  if el[8][int(el[2])+1][1] == 'SCONJ':
    sconjs.append(el)

In [86]:
sconjs = pandas.DataFrame(sconjs)
sconjs.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
sconjs = sconjs.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [87]:
#for el in sconjs.values:
#  change_relative_comma(el)

In [88]:
commas_delete = commas_delete[commas_delete['corrected_text'].apply(lambda x: x not in sconjs['corrected_text'].values)]

In [89]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in sconjs['corrected_text'].values)]

In [90]:
def others_delete_commas(sample):

  tag = 'punctuation'
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  i = list(sample[7].keys())[0]
  index1 = list(sample[7].keys())[0]
  while i != int(sample[2])-1:
    i += 1
    if sample[7][i] == ',':
      index1 = i +1

  index2 = int(sample[2]) + 1
  indexes = range(index1, index2)

  error_area = []
  for b in indexes:
    error_area.append(str(sample[7][b]))

  pprint('CORRECTION: ' + ' '.join(error_area[0:-1]))
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

In [91]:
#for el in commas_delete.values:
#  others_delete_commas(el)

In [92]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in commas_delete['corrected_text'].values)]

In [93]:
adding_commas = []
for el in punctuation.values:
  if str(el[4])[-1] != ',':
    if '.' not in str(el[4]):
      if str(el[4])[-1] != '.':
        if str(el[5]).endswith(','):
          adding_commas.append(el)

## Добавление запятых

Вводные слова

In [94]:
adding_commas = pandas.DataFrame(adding_commas)
adding_commas.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
adding_commas = adding_commas.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [95]:
first_constructions = ['Besides that', 'Besides', 'To my mind', 'On the one hand', 'On the other hand', 'As a result', 'As a result of', 'As a consequence', 'As a consequence of', 
                       'First of all', 'First and foremost', 'Last but not least', 'Finally', 'For instance', 'For example', 'For this reason', 'Hence', 'Consequently', 'To sum up', 
                       'To conclude', 'To begin with', 'To start with', 'In addition', 'In addition to that', 'In addition to', 'At the same time', 'In conclusion', 'All in all', 'As can be seen', 
                       'As we can see', 'According to', 'Due to', 'Thanks to', 'What is more', 'Speaking about', 'Speaking of', 'Moreover', 'Furthermore', 'However', 'Thus', 'Therefore', 
                       'In contrast to', 'In contrast', 'Taking into consideration', 'Finally', 'To make the long story short', 'To cut the long story short', 'Surprisingly', 'Unsurprisingly', 
                       'Undoubtedly', 'Overwhelmingly', 'On rare occasions', 'Infrequently', 'In spite of', 'Despite', 'Having said that', 'As if that was', 'As if it was', 'As if this was', 
                       'To tell the truth', 'To be honest', 'To be fair', 'In truth', 'In all honesty', 'In all fairness', 'In all likelihood', 'In all seriousness', 'Without loss of generality', 
                       'With all my heart', 'With due respect', 'With all due respect', 'Without any reservation', 'Without further ado', 'Without any ado', 'Without any doubt', 'Without a doubt',
                       'Undoubtedly', 'Unanimously', 'Inadvertently', 'By chance', 'By accident', 'Just by accident', 'Just by chance' 'In the nick of time', 'In a jiffy', 'At the last minute', 
                       'At the last moment', 'At the earliest convenience', 'At the earliest opportunity', 'At the earliest possible time','Firstly','Secondly','Thirdly', 'As an example', 'In fact',
                       'This is why', 'In some way', 'By the way', 'In this way', 'So that way', 'So this way', 'From my point of view', 'For this point of view', 'And from this view', 'In my view',
                       'For many years', 'Also', 'Meanwhile', 'In our progressive world','So','In other words', 'In that case', 'In this case', 'In these cases', 'In general', 'Further',
                       'Of course', 'These days', 'Generally', 'Actually', 'For me', 'Overall', 'Nevertheless', 'Namely', 'Otherwise', 'Clearly','Futhermore','Recently','From my perspective', 
                       'Personally', 'Probably', 'Afterwards', 'Later', 'Now', 'Then', 'There', 'Inspite of that', 'Nowadays']

In [96]:
introductory_constructions = []
for k in first_constructions:
  for el in adding_commas.values:
    sample = el[9].split(',')[0]
    if k == sample:
      if len(el[6].split(k)) == 2:
        if el[6].split(k)[1].startswith(','):
          pass
        else:
          introductory_constructions.append(el)
    else:
      pass

In [97]:
introductory_constructions = pandas.DataFrame(introductory_constructions)
introductory_constructions.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
introductory_constructions = introductory_constructions.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [98]:
def change_introductory_constructions_1(sample):
  tag = 'punctuation'
  index = int(sample[2])

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  
  indexes = list(range(int(list(sample[7].keys())[0]),int(sample[2])+1))

  pprint('CORRECTION: ' + str(sample[9].split(',')[0]) + ',')

  error_area = [] # при внедрении в хептабот стоит пользоваться только этими индексами для выделения области ошибки
  for el in indexes:
    error_area.append(sample[7][el])
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

In [99]:
#for el in introductory_constructions.values:
#  change_introductory_constructions_1(el)

In [100]:
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in introductory_constructions['corrected_text'].values)]

In [101]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in introductory_constructions['corrected_text'].values)]

In [102]:
introductory2 = [i for i in first_constructions if i != 'There']

In [103]:
introductory_inside = []
for i in introductory2:
  for el in adding_commas.values:
    if str(el[4]) == i.lower():
      if str(el[4]) + ',' == str(el[5]):
        introductory_inside.append(el)
    elif str(el[7][int(el[2])+1]) == i.lower():
      introductory_inside.append(el)

In [104]:
introductory_inside = pandas.DataFrame(introductory_inside)
introductory_inside.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
introductory_inside = introductory_inside.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [105]:
def change_introductory_inside(sample):
  tag = 'punctuation'
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))

  for i in introductory2:
    if str(sample[4]) == i.lower():
        if str(el[4]) + ',' == str(el[5]):

          i = list(sample[7].keys())[0]
          index1 = list(sample[7].keys())[0]
          while i != el[2]:
            i += 1
            if el[7][i] == ',':
              index1 = i + 1

          index2 = int(el[2]) + 1
          indexes = range(index1, index2)

          error_area = []
          for b in indexes:
            error_area.append(str(el[7][b]))

          pprint('CORRECTION: ' + ' '.join(error_area) + ',')
          pprint('ERROR AREA: '+ ' '.join(error_area))
          pprint('TAG: ' + tag)

    elif str(sample[7][int(sample[2])+1]) == i.lower():
      index =int(sample[2])+1
      
      pprint('CORRECTION: ' + str(sample[7][index]) + ',')
      pprint('ERROR AREA: '+ str(sample[7][index]))
      pprint('TAG: ' + tag)
  

In [106]:
#for el in introductory_inside.values:
#  change_introductory_inside(el)

In [107]:
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in introductory_inside['corrected_text'].values)]

In [108]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in introductory_inside['corrected_text'].values)]

In [109]:
in_modern_world_adding_commas = []
for el in adding_commas.values:
  sample = el[9].split(',')
  try:
    if sample[0] == 'In modern world':
      if len(el[6].split(',')[0].split()) > 3:
        in_modern_world_adding_commas.append(el)
    elif sample[0] == 'In the modern world':
      if len(el[6].split(',')[0].split()) > 4:
        in_modern_world_adding_commas.append(el)
    elif sample[0] == 'In a modern world':
      if len(el[6].split(',')[0].split()) > 4:
        in_modern_world_adding_commas.append(el)
    elif sample[1] == ' in a modern world':
      if len(el[6].split(',')[1].split()) > 4:
          in_modern_world_adding_commas.append(el)
    elif sample[1] == ' in the modern world':
      if len(el[6].split(',')[1].split()) > 4:
          in_modern_world_adding_commas.append(el)
    elif sample[1] == ' in modern world':
      if len(el[6].split(',')[1].split()) > 3:
          in_modern_world_adding_commas.append(el)
    elif sample[1] == ' in our modern world':
      if len(el[6].split(',')[1].split()) > 4:
          in_modern_world_adding_commas.append(el)
    elif sample[1] == ' In our modern world':
      if len(el[6].split(',')[1].split()) > 4:
          in_modern_world_adding_commas.append(el)     
    else:
      pass
  except:
    pass

In [110]:
in_modern_world_adding_commas = pandas.DataFrame(in_modern_world_adding_commas)
in_modern_world_adding_commas.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
in_modern_world_adding_commas = in_modern_world_adding_commas.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [111]:
def change_modern_world_add_comma(sample):

  tag = 'punctuation'

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  

  i = list(sample[7].keys())[0]
  index1 = list(sample[7].keys())[0]
  while i != el[2]:
    i += 1
    if el[7][i] == ',':
      index1 = i + 1

  index2 = int(el[2]) + 1
  indexes = range(index1, index2)

  error_area = []
  for b in indexes:
    error_area.append(str(el[7][b]))

  pprint('CORRECTION: ' + ' '.join(error_area) + ',')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

In [112]:
#for el in in_modern_world_adding_commas.values:
#  change_modern_world_add_comma(el)

In [113]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in in_modern_world_adding_commas['corrected_text'].values)]

In [114]:
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in in_modern_world_adding_commas['corrected_text'].values)]

In [115]:
gerund_after_commas = []
for el in adding_commas.values:
  try:
    element = str(el[4])
    f = re.findall(r'[a-zA-Z]+', element)
    a = re.findall(r'\d+', element)
    k = re.findall(r'\W', element)
    all = f+a+k
    if ' ' in all:
      all.remove(' ')

    length = len(all)
    index2 = int(el[2]) + int(length) 

    if el[8][index2][2] == 'VBG':
      gerund_after_commas.append(el)

  except:
    pass

In [116]:
gerund_after_commas = pandas.DataFrame(gerund_after_commas)
gerund_after_commas.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
gerund_after_commas = gerund_after_commas.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [117]:
#for el in gerund_after_commas.values:
#  change_modern_world_add_comma(el)

In [118]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in gerund_after_commas['corrected_text'].values)]

In [119]:
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in gerund_after_commas['corrected_text'].values)]

In [120]:
relative_clauses2 = []
for el in adding_commas.values:
  try:
    element = str(el[4])
    ind = int(el[2])
    f = re.findall(r'[a-zA-Z]+', element)
    a = re.findall(r'\d+', element)
    k = re.findall(r'\W', element)
    all = f+a+k
    if ' ' in all:
      all.remove(' ')

    length = len(all)
    index = int(el[2]) + int(length)


    if el[8][index][0] == 'which':
      relative_clauses2.append(el)
    elif el[8][index][0] == 'who':
      relative_clauses2.append(el)


    elif el[7][ind+1] == 'which':
      relative_clauses2.append(el)
    elif el[7][ind+1] == 'who':
      relative_clauses2.append(el)
  except:
    pass

In [121]:
relative_clauses2 = pandas.DataFrame(relative_clauses2)
relative_clauses2.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
relative_clauses2 = relative_clauses2.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [122]:
def change_relative(sample):
  tag = 'relative clause'

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))

  element = str(sample[4])
  f = re.findall(r'[a-zA-Z]+', element)
  a = re.findall(r'\d+', element)
  k = re.findall(r'\W', element)
  all = f+a+k
  
  if ' ' in all:
    all.remove(' ')

  length = len(all)
  index = int(sample[2]) + int(length) 

  pprint('CORRECTION: ' + ',' + ' ' + str(sample[7][index]))
  pprint('ERROR AREA: '+ str(sample[7][index]))
  pprint('TAG: ' + tag)

In [123]:
#for el in relative_clauses2.values:
 # change_relative(el)

In [124]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in relative_clauses2['corrected_text'].values)]

In [125]:
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in relative_clauses2['corrected_text'].values)]

In [126]:
that_clauses = []
for el in adding_commas.values:
  index = int(el[2])

  if el[7][index+1] == 'that':
    that_clauses.append(el)

In [127]:
that_clauses = pandas.DataFrame(that_clauses)
that_clauses.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
that_clauses = that_clauses.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [128]:
def change_that(sample):
  tag = 'relative clause'
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))

  element = str(sample[4])
  f = re.findall(r'[a-zA-Z]+', element)
  a = re.findall(r'\d+', element)
  k = re.findall(r'\W', element)
  all = f+a+k

  if ' ' in all:
    all.remove(' ')
  
  
  length = len(all)

  ind = int(sample[2]) + int(length)
  #nsubj
  if sample[8][ind][3] == 'nsubj':
    i = list(sample[7].keys())[0]
    index1 = list(sample[7].keys())[0]
    while i != el[2]:
      i += 1
      if el[7][i] == ',':
        index1 = i + 1

    index2 = int(el[2]) + 1
    indexes = range(index1, index2)

    error_area = []
    for b in indexes:
      error_area.append(str(el[7][b]))
  

    pprint('CORRECTION: ' + ' '.join(error_area) + ',')
    pprint('ERROR AREA: '+ ' '.join(error_area))

  #dobj
  elif sample[8][ind][3] == 'dobj':
    print('the comma is not needed')

  #det
  elif sample[8][ind][3] == 'det':
    i = list(sample[7].keys())[0]
    index1 = list(sample[7].keys())[0]
    while i != el[2]:
      i += 1
      if el[7][i] == ',':
        index1 = i + 1

    index2 = int(el[2]) + 1
    indexes = range(index1, index2)

    error_area = []
    for b in indexes:
      error_area.append(str(el[7][b]))
    
    pprint('CORRECTION: ' + ' '.join(error_area) + ',')
    pprint('ERROR AREA: '+ ' '.join(error_area))
    
  # prep
  elif sample[8][ind][3] == 'prep':
    error_area = sample[4]
    correction = sample[5]

    pprint('CORRECTION: ' + correction)
    pprint('ERROR AREA: '+ error_area)
    
  elif sample[8][ind][3] == 'mark':
    print('strange thing')

  print('\n')

In [129]:
#for el in that_clauses.values:
#  change_that(el)

In [130]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in that_clauses['corrected_text'].values)]

In [131]:
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in that_clauses['corrected_text'].values)]

In [132]:
first_gerunds = []
for el in adding_commas.values:
  gerunds = []
  index1 = list(el[8].keys())[0]
  index2 = int(el[2])
  indexes = range(index1, index2)
  for ind in indexes:
    gerunds.append(el[-1][ind][-1])

  if el[-1][index1][2] == 'VBG':
    if el[-1][index1][3] != 'csubj':
      if el[6].split(',')[0] != el[9].split(',')[0]:
        if 'nsubj' not in gerunds:
          first_gerunds.append(el)

In [133]:
first_gerunds = pandas.DataFrame(first_gerunds)
first_gerunds.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
first_gerunds = first_gerunds.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [134]:
def change_gerund(sample):
  tag = 'punctuation'

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  
  error_area = []
  for k, v in sample[7].items():
    if k != int(sample[2])+1:
      error_area.append(str(v))
    else:
      break

  pprint('CORRECTION: ' + ' '.join(error_area) + ',')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

In [135]:
#for el in first_gerunds.values:
 # change_gerund(el)

In [136]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in first_gerunds['corrected_text'].values)]

In [137]:
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in first_gerunds['corrected_text'].values)]

In [138]:
conjuncs = ['and', 'where', 'while', 'so', 'wherears', 'but', 'even', 'although', 'as', 'then', 'whether', 'like', 'since', 'because', 'when', 'whilst', 'regardless', 'especially']

In [139]:
conjuncs_df = []
for conj in conjuncs:
  try:
    for el in adding_commas.values:
      element = str(el[4])
      f = re.findall(r'[a-zA-Z]+', element)
      j = re.findall(r'\d+[,.]\d+',element)
      l = re.findall(r'[%]+', element)
      change = re.findall(r"[\w']+|[.,!?;()%-]", str(element))
      if len(j) == 0:
        all = change
      else:

        all = f+j+l

      if ' ' in all:
        all.remove(' ')
      
      length = len(all)

      if el[7][int(el[2])+length] == conj:
        if el[7][int(el[2])+length+1] != 'of':
          conjuncs_df.append(el)
  except:
    pass

In [140]:
conjuncs_df = pandas.DataFrame(conjuncs_df)
conjuncs_df.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
conjuncs_df = conjuncs_df.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [141]:
def change_conj(sample):
  tag = 'punctuation'
  marks = string.punctuation 
  if str(sample[4]) + ',' != str(sample[5]):
    if str(sample[4]) in marks:
      pprint("USER'S TEXT: " + sample[6])
      pprint('CORRECTED TEXT: ' + str(sample[9]))
      
      error_area = sample[7][int(sample[2])] + ' ' + sample[7][int(sample[2]+1)] 
      correction = sample[10][int(sample[2])] + ' ' + sample[10][int(sample[2]+1)] 

      pprint('CORRECTION: ' + correction)
      pprint('ERROR AREA: '+ error_area)
      pprint('TAG: ' + tag)

  else:
    pprint("USER'S TEXT: " + sample[6])
    pprint('CORRECTED TEXT: ' + str(sample[9]))
    
    error_area = sample[7][int(sample[2]+1)]

    pprint('CORRECTION: ' + ', ' + error_area)
    pprint('ERROR AREA: '+ error_area)
    pprint('TAG: ' + tag)

In [142]:
#for el in conjuncs_df.values:
#  change_conj(el)

In [143]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in conjuncs_df['corrected_text'].values)]

In [144]:
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in conjuncs_df['corrected_text'].values)]

In [145]:
conjuncs_df2 = []
for el in adding_commas.values:
  try:
    element = str(el[4])
    f = re.findall(r'[a-zA-Z]+', element)
    j = re.findall(r'\d+[,.]\d+',element)
    l = re.findall(r'[%]+', element)
    change = re.findall(r"[\w']+|[.,!?;()%-]", str(element))
    if len(j) == 0:
      all = change
    else:

      all = f+j+l

    if ' ' in all:
      all.remove(' ')
    
    length = len(all)

    if el[7][int(el[2])+length] == 'such':
      if el[7][int(el[2])+length+1] == 'as':
        conjuncs_df2.append(el)
    elif el[7][int(el[2])+length] == 'because':
      if el[7][int(el[2])+length+1] == 'of':
        conjuncs_df2.append(el)
    elif el[7][int(el[2])+length] == 'of':
      if el[7][int(el[2])+length+1] == 'course':
        conjuncs_df2.append(el)
  except:
    pass

In [146]:
conjuncs_df2 = pandas.DataFrame(conjuncs_df2)
conjuncs_df2.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
conjuncs_df2 = conjuncs_df2.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [147]:
def change_conj2(sample):
  tag = 'punctuation'
  marks = string.punctuation 
  if str(sample[4]) + ',' != str(sample[5]):
    if str(sample[4]) in marks:
      pprint("USER'S TEXT: " + sample[6])
      pprint('CORRECTED TEXT: ' + str(sample[9]))
      indexes = [sample[7][int(sample[2])] , sample[7][int(sample[2]+1)] , sample[7][int(sample[2]+2)] ]
      
      error_area = sample[7][int(sample[2])] + ' ' + sample[7][int(sample[2]+1)] + ' ' + sample[7][int(sample[2]+2)] 
      correction = sample[10][int(sample[2])] + ' ' + sample[10][int(sample[2]+1)] + ' ' + sample[10][int(sample[2]+2)] 

      pprint('CORRECTION: ' + correction)
      pprint('ERROR AREA: '+ error_area)
      pprint('TAG: ' + tag)

  else:
    pprint("USER'S TEXT: " + sample[6])
    pprint('CORRECTED TEXT: ' + str(sample[9]))
    indexes = [sample[7][int(sample[2]+1)], sample[7][int(sample[2]+2)] ]
    
    error_area =  sample[7][int(sample[2]+1)] + ' ' + sample[7][int(sample[2]+2)] 

    pprint('CORRECTION: ' + ', ' + error_area)
    pprint('ERROR AREA: '+ error_area)
    pprint('TAG: ' + tag)

In [148]:
#for el in conjuncs_df2.values:
 # change_conj2(el)

In [149]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in conjuncs_df2['corrected_text'].values)]

In [150]:
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in conjuncs_df2['corrected_text'].values)]

In [151]:
preps2 = []
for el in adding_commas.values:
  if el[6].split(',')[0] != el[9].split(',')[0]:

    if len(el[9].split(',')[0].split()) < 9:
      index1 = list(el[8].keys())[0]
      index2 = int(el[2])
      indexes = range(index1,index2)
      preps_list = []
      for ind in indexes:  
        preps_list.append(el[8][ind][3])


      if 'prep' in preps_list:
        preps2.append(el)


In [152]:
preps2 = pandas.DataFrame(preps2)
preps2.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
preps2 = preps2.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [153]:
def change_preps2(sample):
  tag = 'punctuation'

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  
  if str(sample[4]).startswith(','):
    if str(sample[5]).endswith(','):
      index1 = int(list(sample[7].keys())[0])
      index2 = int(sample[2]) + 2
      indexes = range(index1, index2)
      error_area = []
      correction = []
      for i in indexes:
        error_area.append(str(sample[7][i]))
        correction.append(str(sample[10][i]))
  
      pprint('CORRECTION: ' + ' '.join(correction[0:-1])+',')
      pprint('ERROR AREA: '+ ' '.join(error_area))
      pprint('TAG: ' + tag)

  else:
    index1 = int(list(sample[7].keys())[0])
    index2 = int(sample[2]) + 1
    indexes = range(index1, index2)
    error_area = []
    for i in indexes:
      error_area.append(str(sample[7][i]))

    pprint('CORRECTION: ' + ' '.join(error_area) + ',')
    pprint('ERROR AREA: '+ ' '.join(error_area))
    pprint('TAG: ' + tag)

In [154]:
#for el in preps2.values:
#  change_preps2(el)

In [155]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in preps2['corrected_text'].values)]

In [156]:
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in preps2['corrected_text'].values)]

In [157]:
ins = []
for el in adding_commas.values:
  ind = list(el[7].keys())[0]
  if el[6].split(',')[0] != el[9].split(',')[0]:
    if len(el[9].split(',')[0].split()) < 6:
      if el[8][ind][0] == 'In':
        ins.append(el)

In [158]:
ins = pandas.DataFrame(ins)
ins.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
ins = ins.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [159]:
#for el in ins.values:
#  change_preps2(el)

In [160]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in ins['corrected_text'].values)]

In [161]:
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in ins['corrected_text'].values)]

In [162]:
first_to_infs = []
for el in adding_commas.values:
  ind = list(el[8].keys())[0]  
  if el[6].split(',')[0] != el[9].split(',')[0]:
    if el[8][ind][0] == 'To':
      if el[8][ind+1][3] == 'advcl':
        first_to_infs.append(el)

In [163]:
first_to_infs = pandas.DataFrame(first_to_infs)
first_to_infs.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
first_to_infs = first_to_infs.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [164]:
def change_first_to(sample):
  tag = 'punctuation'
  index1 = int(list(sample[7].keys())[0])
  index2 = int(sample[2]) + 1
  indexes = range(index1, index2)

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
      
  
  error_area = []
  for i in indexes:
    error_area.append(str(sample[7][i]))

  pprint('CORRECTION: ' + ' '.join(error_area) + ',')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

In [165]:
#for el in first_to_infs.values:
#  change_first_to(el)

In [166]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in first_to_infs['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in first_to_infs['corrected_text'].values)]

In [167]:
commas_dots2 = []
for el in adding_commas.values:
  if str(el[4]).startswith(','):
    if str(el[5]).startswith('.'):
      if str(el[5])[2].islower() == False:
        commas_dots2.append(el)

In [168]:
commas_dots2 = pandas.DataFrame(commas_dots2)
commas_dots2.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
commas_dots2 = commas_dots2.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [169]:
def change_commas_dots2(sample):
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  tag = 'punctuation'

  i = list(sample[7].keys())[0]
  index1 = list(sample[7].keys())[0]
  while i != el[2]:
    i += 1
    if el[7][i] == ',':
      index1 = i 

  index2 = int(el[2]) + 2
  indexes = range(index1, index2)

  error_area = []
  for b in indexes:
    error_area.append(str(el[7][b]))


  pprint('CORRECTION: ' + '. ' + ' '.join(error_area[1:]).capitalize())
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)


  pprint('CORRECTION: ' + str(sample[5][2:]))
  pprint('ERROR AREA: '+ str(sample[4][2:]))
  pprint('TAG: ' + tag)

In [170]:
#for el in commas_dots2.values:
#  change_commas_dots2(el)

In [171]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in commas_dots2['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in commas_dots2['corrected_text'].values)]

In [172]:
gerund_constructions = []
for el in adding_commas.values:
  index1 = list(el[7].keys())[0]
  index2 = int(el[2])
  i = int(el[2])
  while i != el[2]:
    i -= 1
    if el[7][i] == ',':
      index1 = i 
      #if i[2] == 'VBG':


  indexes = range(index1, index2)
  poss_gerunds = []
  for i in indexes:
    poss_gerunds.append(el[8][i][2])

  if 'VBG' in poss_gerunds:
    gerund_constructions.append(el)

In [173]:
gerund_constructions = pandas.DataFrame(gerund_constructions)
gerund_constructions.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
gerund_constructions = gerund_constructions.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [174]:
def change_gerund_constructions(sample):
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  tag = 'punctuation'

  i = list(sample[7].keys())[0]
  index1 = list(sample[7].keys())[0]
  while i != el[2]:
    i += 1
    if el[7][i] == ',':
      index1 = i + 1

  index2 = int(el[2]) + 1
  indexes = range(index1, index2)

  error_area = []
  for b in indexes:
    error_area.append(str(el[7][b]))


  pprint('CORRECTION: ' + ' '.join(error_area) + ',')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

In [175]:
#for el in gerund_constructions.values:
#  change_gerund_constructions(el)

In [176]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in gerund_constructions['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in gerund_constructions['corrected_text'].values)]

In [177]:
opinions = []
for el in adding_commas.values:
  if str(el[4]) == 'opinion':
    if str(el[4]) + ',' == str(el[5]):
      opinions.append(el)

In [178]:
opinions = pandas.DataFrame(opinions)
opinions.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
opinions = opinions.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [179]:
#for el in opinions.values:
#  change_gerund_constructions(el)

In [180]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in opinions['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in opinions['corrected_text'].values)]

In [181]:
cases = []
for el in adding_commas.values:
  if str(el[4]) == 'case':
    if str(el[4]) + ',' == str(el[5]):
      cases.append(el)
  elif str(el[4]) == 'cases':
    if str(el[4]) + ',' == str(el[5]):
      cases.append(el)

In [182]:
cases = pandas.DataFrame(cases)
cases.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
cases = cases.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [183]:
#for el in cases.values:
 # change_gerund_constructions(el)

In [184]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in cases['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in cases['corrected_text'].values)]

In [185]:
verbs_after_correction = []
for el in adding_commas.values:
  if str(el[4]) + ',' == str(el[5]):
    element = str(el[4])
    f = re.findall(r'[a-zA-Z]+', element)
    a = re.findall(r'\d+', element)
    k = re.findall(r'\W', element)
    all = f+a+k
    if ' ' in all:
      all.remove(' ')

    length = len(all)
    index = int(el[2]) + int(length)

    if el[8][index][2] == 'VBD':
      verbs_after_correction.append(el)

In [186]:
verbs_after_correction = pandas.DataFrame(verbs_after_correction)
verbs_after_correction.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
verbs_after_correction = verbs_after_correction.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [187]:
#for el in verbs_after_correction.values:
#  change_gerund_constructions(el)

In [188]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in verbs_after_correction['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in verbs_after_correction['corrected_text'].values)]

In [189]:
ifs = []
for el in adding_commas.values:
  if str(el[4]) + ',' == str(el[5]):
    element = str(el[4])
    f = re.findall(r'[a-zA-Z]+', element)
    a = re.findall(r'\d+', element)
    k = re.findall(r'\W', element)
    all = f+a+k
    if ' ' in all:
      all.remove(' ')

    length = len(all)

    index = int(el[2]) + int(length)

    if el[8][index][0] == 'if':
      ifs.append(el)
    elif el[8][index+1][0] == 'if':
      ifs.append(el)
    else:
      index1 = list(el[7].keys())[0]
      index2 = int(el[2])
      i = int(el[2])
      while i != el[2]:
        i -= 1
        if el[7][i] == ',':
          index1 = i - 1
          #if i[2] == 'VBG':


      indexes = range(index1, index2+1)
      poss_ifs = []
      for i in indexes:
        poss_ifs.append(el[8][i][0])

      if poss_ifs[0] == 'If':
        ifs.append(el)
      else:
        try:
          index1 = list(el[7].keys())[0]
          index2 = int(el[2])
          i = int(el[2])
          while i != index1:
            i -= 1
            if el[7][i] == ',':
              index1 = i 
              
              #if i[2] == 'VBG':


          indexes = range(index1+1, index2+1)
          poss_ifs = []
          for i in indexes:
            poss_ifs.append(el[8][i][0])

          if poss_ifs[0] == 'if':
            ifs.append(el)
        except:
          pass


In [190]:
ifs = pandas.DataFrame(ifs)
ifs.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
ifs = ifs.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [191]:
#for el in ifs.values:
#  change_gerund_constructions(el)

In [192]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in ifs['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in ifs['corrected_text'].values)]

In [193]:
pass_commas = []
for el in adding_commas.values:
  if str(el[4]) == '-':
    if str(el[5]) == ',':
      pass_commas.append(el)

In [194]:
pass_commas = pandas.DataFrame(pass_commas)
pass_commas.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
pass_commas = pass_commas.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [195]:
def change_pass_commas(sample):
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  tag = 'punctuation'

  i = list(sample[7].keys())[0]
  index1 = list(sample[7].keys())[0]
  while i != el[2]:
    i += 1
    if el[7][i] == ',':
      index1 = i + 1

  index2 = int(el[2]) + 1
  indexes = range(index1, index2)

  error_area = []
  for b in indexes:
    error_area.append(str(el[7][b]))


  pprint('CORRECTION: ' + ' '.join(error_area[0:-1]) + ',')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

In [196]:
#for el in pass_commas.values:
#  change_pass_commas(el)

In [197]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in pass_commas['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in pass_commas['corrected_text'].values)]

In [198]:
colon_commas = []
for el in adding_commas.values:
  if str(el[4]) == ':':
    if str(el[5]) == ',':
      colon_commas.append(el)

In [199]:
colon_commas = pandas.DataFrame(colon_commas)
colon_commas.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
colon_commas = colon_commas.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [200]:
#for el in colon_commas.values:
#  change_pass_commas(el)

In [201]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in colon_commas['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in colon_commas['corrected_text'].values)]

In [202]:
preps2 = []
for el in adding_commas.values:
  i = int(el[2])
  try:
    if str(el[4]) == el[8][i][0]:
      if el[8][i-1][3] == 'prep':
        preps2.append(el)
      elif el[8][i-2][3] == 'prep':
        preps2.append(el)
      elif el[8][i-3][3] == 'prep':
        preps2.append(el)
      elif el[8][i-4][3] == 'prep':
        preps2.append(el)
      elif el[8][i-5][3] == 'prep':
        preps2.append(el)
  except:
    pass



In [203]:
preps2 = pandas.DataFrame(preps2)
preps2.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
preps2 = preps2.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [204]:
#for el in preps2.values:
  #change_gerund_constructions(el)

In [205]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in preps2['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in preps2['corrected_text'].values)]

In [206]:
agos = []
for el in adding_commas.values:
  if str(el[4]) == 'ago':
    if str(el[4]) +',' == str(el[5]):
      agos.append(el)

In [207]:
agos = pandas.DataFrame(agos)
agos.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
agos = agos.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [208]:
#for el in agos.values:
#  change_gerund_constructions(el)

In [209]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in agos['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in agos['corrected_text'].values)]

In [210]:
thats = []
for el in adding_commas.values:
  if str(el[4]) == 'that':
    if str(el[4]) +',' == str(el[5]):
      thats.append(el)

In [211]:
thats = pandas.DataFrame(thats)
thats.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
thats = thats.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [212]:
#for el in thats.values:
#  change_gerund_constructions(el)

In [213]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in thats['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in thats['corrected_text'].values)]

In [214]:
preps3 = []
for el in adding_commas.values:
  if str(el[4]) + ',' == str(el[5]):
    element = str(el[4])
    f = re.findall(r'[a-zA-Z]+', element)
    a = re.findall(r'\d+', element)
    k = re.findall(r'\W', element)
    all = f+a+k
    if ' ' in all:
      all.remove(' ')

    length = len(all)

    index = int(el[2]) + int(length)

    if el[8][index][3] == 'prep':
      preps3.append(el)
    elif el[8][index+1][3] == 'prep':
      preps3.append(el)
    else:

      i = int(el[2])
      index1 = list(el[7].keys())[0]
      index2 = int(el[2])
      try:
        if el[8][i][0] !=',':
          while el[8][i][3] != 'prep':
            i -= 1
            index1 = i





        indexes = range(index1, index2)
        poss_ifs = []
        for i in indexes:
          poss_ifs.append(el[8][i][3])
          

          if poss_ifs[0] == 'prep':
            preps3.append(el)
          elif 'prep' in poss_ifs:
            preps3.append(el)
      
      except:
        pass


In [215]:
preps3 = pandas.DataFrame(preps3)
preps3.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
preps3 = preps3.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [216]:
def change_preps_inside(sample):
  tag = 'punctuation'

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  index = int(el[2])
  if el[8][index][3] == 'prep':
    index1 = int(el[2])
    error_area = el[7][index1]

    pprint('CORRECTION: ' + str(error_area) + ',')
    pprint('ERROR AREA: '+ str(error_area))
    pprint('TAG: ' + tag)
  elif el[8][index+1][3] == 'prep':
    index1 = int(el[2]) + 1
    error_area = el[7][int(el[2]) + 1]

    pprint('CORRECTION: ' + ', ' + str(error_area))
    pprint('ERROR AREA: '+ str(error_area))
    pprint('TAG: ' + tag)

  elif el[8][index+2][3] == 'prep':
    index1 = int(el[2]) + 2
    error_area = el[7][int(el[2]) + 1]

    pprint('CORRECTION: ' + ', ' + str(error_area))
    pprint('ERROR AREA: '+ str(error_area))
    pprint('TAG: ' + tag)

  else:
    i = int(el[2])
    index1 = list(el[7].keys())[0]
    index2 = int(el[2])
    while el[8][i][3] != 'prep':
      i -= 1
      index1 = i 





    indexes = range(index1, index2+1)
    error_area = []
    for i in indexes:
      error_area.append(el[7][i])
    pprint('CORRECTION: ' + ' '.join(error_area) + ',')
    pprint('ERROR AREA: '+ ' '.join(error_area))
    pprint('TAG: ' + tag)

  


In [217]:
#for el in preps3.values:
#  change_preps_inside(el)

In [218]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in preps3['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in preps3['corrected_text'].values)]

In [219]:
other_cases1 = []
for el in adding_commas.values:
  if (str(el[4])) +',' == str(el[5]):
    other_cases1.append(el)


In [220]:
other_cases1 = pandas.DataFrame(other_cases1)
other_cases1.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
other_cases1 = other_cases1.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [221]:
#for el in other_cases1.values:
#  change_gerund_constructions(el)

In [222]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in other_cases1['corrected_text'].values)]
adding_commas = adding_commas[adding_commas['corrected_text'].apply(lambda x: x not in other_cases1['corrected_text'].values)]

In [223]:
def change_other_cases2(sample):
  tag = 'punctuation'
  element = str(el[4])
  f = re.findall(r'[a-zA-Z]+', element)
  a = re.findall(r'\d+', element)
  k = re.findall(r'\W', element)
  all = f+a+k
  if ' ' in all:
    all.remove(' ')

  length = len(all)

  index1 = int(el[2])
  index2 = int(el[2]) + int(length)

  indexes = range(index1, index2)
  error_area = []
  for i in indexes:
    error_area.append(el[7][i])

  pprint('CORRECTION: ' + str(sample[5]))
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

In [224]:
#for el in adding_commas.values:
#  change_other_cases2(el)

In [225]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in adding_commas['corrected_text'].values)]

In [226]:
dots_end = []
for el in punctuation.values:
  for i in string.punctuation:
    if i not in str(el[4]):
      if str(el[5]).endswith('.'):
        dots_end.append(el)

In [227]:
dots_end = pandas.DataFrame(dots_end)
dots_end.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
dots_end = dots_end.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [228]:
def change_dots_end(sample):
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  tag = 'punctuation'

  element = str(el[4])
  f = re.findall(r'[a-zA-Z]+', element)
  a = re.findall(r'\d+', element)
  k = re.findall(r'\W', element)
  all = f+a+k
  if ' ' in all:
    all.remove(' ')

  length = len(all)


  i = list(sample[7].keys())[0]
  index1 = list(sample[7].keys())[0]
  while i != el[2]:
    i += 1
    if el[7][i] == ',':
      index1 = i + 1

  index2 = int(el[2]) 
  indexes = range(index1, index2+length)

  error_area = []
  for b in indexes:
    error_area.append(str(el[7][b]))


  pprint('CORRECTION: ' + ' '.join(error_area) + '.')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

In [229]:
#for el in dots_end.values:
#  change_dots_end(el)

In [230]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in dots_end['corrected_text'].values)]

In [231]:
poor_cases = []
for el in punctuation.values:
  if isinstance(el[4],float):
    el4 = str(int(el[4]))
  else:
    el4 = str(el[4])
  if el4 == str(el[5]):
    poor_cases.append(el)

In [232]:
poor_cases = pandas.DataFrame(poor_cases)
poor_cases.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
poor_cases = poor_cases.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [233]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in poor_cases['corrected_text'].values)]

In [234]:
dots_start = []
for el in punctuation.values:
  element = str(el[4])
  f = re.findall(r'[a-zA-Z]+', element)
  a = re.findall(r'\d+', element)
  k = re.findall(r'\W', element)
  all = f+a+k
  if ' ' in all:
    all.remove(' ')

  if len(all) == 1:
    if all[0].isalpha():
      if str(el[5]).startswith('.'):
          dots_start.append(el)

In [235]:
dots_start = pandas.DataFrame(dots_start)
dots_start.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
dots_start = dots_start.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [236]:
def change_dots2(sample):

  tag = 'punctuation'
  parts_of_sentences = ['ROOT', 'relcl', 'nsubj', 'expl', 'conj', 'ccomp', 'aux', 
                      'neg', 'nsubjpass','punct','intj', 'acl']
  
  all_sents = []
  first_sent = []
  for el in sample[11].values():
    if el[-1] in parts_of_sentences:
      all_sents.append(el)

  for i in all_sents:
    first_sent.append(i)
    if i[0] == '.':
      ind = all_sents.index(i)
      break
      
  second_sent = all_sents[ind::]
  del first_sent[-1]

  if len(first_sent) >= 4:
    pass

  else:
    parts_of_sentences2 = ['nsubj','ROOT', 'expl']
    sent = []
    for el in first_sent:
      if el in parts_of_sentences2:
        sent.append(el)
        if len(sent) >= 2:
          pass
        else:
          'the comma is not needed'

  index1 = int(list(sample[8].keys())[0])
  index2 = int(sample[2]) + 1
  indexes = range(index1, index2)
  error_area = [sample[7][i] for i in indexes]

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))



  if len(str(sample[4]).split()) == len(str(sample[5]).split()):
    error_area = error_area[0:-1]
    pprint('CORRECTION: ' + ' '.join(error_area) + '.')
    pprint('ERROR AREA: '+ ' '.join(error_area))
    pprint('TAG: ' + tag)

  elif '. '+str(sample[4]).lower() == str(sample[5]).lower():
    error_area = error_area[0:-1]
    pprint('CORRECTION: ' + ' '.join(error_area) + '.')
    pprint('ERROR AREA: '+ ' '.join(error_area))
    pprint('TAG: ' + tag)

  else:
    error_area = error_area[0:-1]
    pprint('CORRECTION: ' + ' '.join(error_area) + '.')
    pprint('ERROR AREA: '+ ' '.join(error_area))
    pprint('TAG: ' + tag)


    if 'a' in str(sample[5]).lower().split() or 'the' in str(sample[5]).lower().split():
      tag = 'articles'
      index1 = int(sample[2]) + 1
      index2 = int(list(sample[7].keys())[-1])
      indexes = range(index1, index2)
      for ind in indexes:
        if sample[8][ind][1] != 'NOUN':
          index2 = ind + 1
          break

      indexes = range(index1, index2+1)
      error_area = []
      for ind in indexes:
        error_area.append(str(sample[7][ind]))

      pprint('CORRECTION: ' + str(sample[5]).split()[1] + ' ' +' '.join(error_area))
      pprint('ERROR AREA: '+ ' '.join(error_area))
      pprint('TAG: ' + tag)

    elif 'a' in str(sample[4]).lower().split() or 'the' in str(sample[4]).lower().split():
      if 'a' not in str(sample[5]).lower().split() or 'the' not in str(sample[5]).lower().split():
        tag = 'articles'
        index1 = int(sample[2]) + 1
        index2 = int(list(sample[7].keys())[-1])
        indexes = range(index1, index2)
        for ind in indexes:
          if sample[8][ind][1] != 'NOUN':
            index2 = ind + 1
            break

        indexes = range(index1, index2+1)
        error_area = []
        for ind in indexes:
          error_area.append(str(sample[7][ind+1]))

        correction = []
        for ind in indexes:
          correction.append(str(sample[10][ind+1]))

        pprint('CORRECTION: ' + ' '.join(correction[0:-1]))
        pprint('ERROR AREA: '+ ' '.join(error_area))
        pprint('TAG: ' + tag)
    else:
      print('HERE ANOTHER CASE!!!!!!')

  print('\n')
    

In [237]:
#for el in dots_start.values:
#  change_dots2(el)

In [238]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in dots_start['corrected_text'].values)]

In [239]:
commas_start = []
for el in punctuation.values:
  element = str(el[4])
  f = re.findall(r'[a-zA-Z]+', element)
  a = re.findall(r'\d+', element)
  k = re.findall(r'\W', element)
  all = f+a+k
  if ' ' in all:
    all.remove(' ')

  if len(all) == 1:
    if all[0].isalpha():
      if str(el[5]).startswith(','):
          commas_start.append(el)

In [240]:
commas_start = pandas.DataFrame(commas_start)
commas_start.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
commas_start = commas_start.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [241]:
def change_commas_start(sample):
  tag = 'punctuation'
  parts_of_sentences = ['ROOT', 'relcl', 'nsubj', 'expl', 'conj', 'ccomp', 'aux', 
                      'neg', 'nsubjpass','punct','intj', 'acl']
  
  all_sents = []
  first_sent = []
  for el in sample[11].values():
    if el[-1] in parts_of_sentences:
      all_sents.append(el)

  for i in all_sents:
    first_sent.append(i)
    if i[0] == '.':
      ind = all_sents.index(i)
      break
      
  second_sent = all_sents[ind::]
  del first_sent[-1]

  if len(first_sent) >= 4:
    pass

  else:
    parts_of_sentences2 = ['nsubj','ROOT', 'expl']
    sent = []
    for el in first_sent:
      if el in parts_of_sentences2:
        sent.append(el)
        if len(sent) >= 2:
          pass
        else:
          'the comma is not needed'

  index1 = int(list(sample[8].keys())[0])
  index2 = int(sample[2]) + 1
  indexes = range(index1, index2)
  error_area = [sample[7][i] for i in indexes]

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))

  error_area = error_area[0:-1]
  pprint('CORRECTION: ' + ' '.join(error_area) + ',')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

  if ', ' + str(sample[4]).lower() == str(sample[5].lower()):
    tag = 'capitalization'
    pprint('CORRECTION: ' + str(sample[5].split()[1]))
    pprint('ERROR AREA: '+  str(sample[4]))
    pprint('TAG: ' + tag)
  elif len(list(set(list(sample[4])) ^ set(list(sample[5])))) < 4:
    tag = 'spelling'
    pprint('CORRECTION: ' + str(sample[5].split()[1]))
    pprint('ERROR AREA: '+  str(sample[4]))
    pprint('TAG: ' + tag)
  elif len(list(set(list(sample[4])) ^ set(list(sample[5])))) > 3:
    tag = 'wrong word'
    pprint('CORRECTION: ' + str(sample[5].split()[1]))
    pprint('ERROR AREA: '+  str(sample[4]))
    pprint('TAG: ' + tag)
  print('\n')

In [242]:
#for el in commas_start.values:
#  change_commas_start(el)

In [243]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in commas_start['corrected_text'].values)]

In [244]:
non_punctuation = []
for el in punctuation.values:
  element = str(el[4])
  element2 = str(el[5])

  f = re.findall(r'[a-zA-Z]+', element)
  a = re.findall(r'\d+', element)
  k = re.findall(r'\W', element)
  all1 = f+a+k
  if ' ' in all1:
    all1.remove(' ')

  f = re.findall(r'[a-zA-Z]+', element2)
  a = re.findall(r'\d+', element2)
  k = re.findall(r'\W', element2)
  all2 = f+a+k
  if ' ' in all2:
    all2.remove(' ')
  

  if ',' not in all1 and '.' not in all1 and ':' not in all1:
    if ',' not in all2 and '.' not in all2 and ':' not in all2:
      non_punctuation.append(el)

In [245]:
non_punctuation = pandas.DataFrame(non_punctuation)
non_punctuation.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
non_punctuation = non_punctuation.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [250]:
def change_nonpunctuation(sample):
  tag='?'
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))

  index = sample[2]
  correction = str(sample[5])
  pprint('CORRECTION: ' + correction)
  pprint('ERROR AREA: '+ str(sample[7][index]))
  pprint('TAG: ' + tag)

In [252]:
#for el in non_punctuation.values:
#  change_nonpunctuation(el)

In [253]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in non_punctuation['corrected_text'].values)]

In [255]:
numerals = []
for el in punctuation.values:
  if isinstance(el[4],float):
    el4 = str(int(el[4]))
  else:
    el4 = str(el[4])
  if el4.isnumeric():
    if str(el[5]).endswith('%'):
      numerals.append(el)

In [256]:
numerals = pandas.DataFrame(numerals)
numerals.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
numerals = numerals.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [258]:
def change_numerals(sample):
  
  tag='numerals'
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))

  index = sample[2]
  correction = str(sample[5])
  pprint('CORRECTION: ' + correction)
  pprint('ERROR AREA: '+ str(sample[7][index]))
  pprint('TAG: ' + tag)

In [260]:
#for el in numerals.values:
#  change_numerals(el)

In [261]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in numerals['corrected_text'].values)]

In [265]:
years2_commas = []
for el in punctuation.values:
  if isinstance(el[4],float):
    el4 = str(int(el[4]))
  else:
    el4 = str(el[4])
  if el4.isnumeric():
    if el4+',' == str(el[5]):
      years2_commas.append(el)

In [266]:
years2_commas = pandas.DataFrame(years2_commas)
years2_commas.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
years2_commas = years2_commas.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [280]:
def change_years2(sample):
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  tag = 'punctuation'


  i = list(sample[7].keys())[0]
  index1 = list(sample[7].keys())[0]
  while i != el[2]:
    i += 1
    if el[7][i] == ',':
      index1 = i + 1

  index2 = int(el[2]) 
  indexes = range(index1, index2 + 1)

  error_area = []
  for b in indexes:
    error_area.append(str(el[7][b]))


  pprint('CORRECTION: ' + ' '.join(error_area) + ',')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

In [282]:
#for el in years2_commas.values:
#  change_years2(el)

In [283]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in years2_commas['corrected_text'].values)]

In [304]:
commas_dot3 = []
for el in punctuation.values:
  try:

    if str(el[4]).startswith(','):
      if str(el[5]).startswith('.'):
        if el[4].split()[1] == el[5].split()[1].lower():
          commas_dot3.append(el)

  except:
    pass

In [305]:
commas_dot3 = pandas.DataFrame(commas_dot3)
commas_dot3.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
commas_dot3 = commas_dot3.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [414]:
def change_commas_dots3(sample):
  tag = 'punctuation'
  parts_of_sentences = ['ROOT', 'relcl', 'nsubj', 'expl', 'conj', 'ccomp', 'aux', 
                      'neg', 'nsubjpass','punct','intj', 'acl']
  
  all_sents = []
  first_sent = []
  for el in sample[11].values():

    if el[-1] in parts_of_sentences:
      all_sents.append(el)

  for i in all_sents:
    first_sent.append(i)
    if i[0] == '.':
      ind = all_sents.index(i)
      break
      
  second_sent = all_sents[ind::]
  del first_sent[-1]

  if len(first_sent) >= 4:
    pass

  else:
    parts_of_sentences2 = ['nsubj','ROOT', 'expl']
    sent = []
    for el in first_sent:
      if el in parts_of_sentences2:
        sent.append(el)
        if len(sent) >= 2:
          pass
        else:
          'the comma is not needed'

  index1 = int(list(sample[8].keys())[0])
  index2 = int(sample[2]) + 1
  indexes = range(index1, index2)
  error_area = [sample[7][i] for i in indexes]

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))

  error_area = error_area[0:-1]
  pprint('CORRECTION: ' + ' '.join(error_area) + '.')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

  if str(sample[4]).endswith('the'):
    element = str(sample[4])


    f = re.findall(r'[a-zA-Z]+', element)
    a = re.findall(r'\d+', element)
    k = re.findall(r'\W', element)
    all = f+a+k
    if ' ' in all:
      all.remove(' ')
    length = len(all)
    
    tag = 'articles'
    index = int(sample[2]) + length - 1
    pprint('CORRECTION: ' +str(sample[7][index]))
    pprint('ERROR AREA: '+  str(sample[7][index-1])+' '+ str(sample[7][index]))
    pprint('TAG: ' + tag)

  print('\n')

In [416]:
#for el in commas_dot3.values:
#  change_commas_dots3(el)

In [331]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in commas_dot3['corrected_text'].values)]

In [338]:
dots_commas = []
for el in punctuation.values:
  try:

    if str(el[4]).startswith('.'):
      if str(el[5]).startswith(','):
        if el[4].split()[1].lower() == el[5].split()[1]:
          dots_commas.append(el)

  except:
    pass

In [339]:
dots_commas = pandas.DataFrame(dots_commas)
dots_commas.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
dots_commas = dots_commas.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [346]:
def change_dots_commas(sample):
  tag = 'punctuation'
  parts_of_sentences = ['ROOT', 'relcl', 'nsubj', 'expl', 'conj', 'ccomp', 'aux', 
                      'neg', 'nsubjpass','punct','intj', 'acl']
  
  all_sents = []
  first_sent = []
  for el in sample[11].values():

    if el[-1] in parts_of_sentences:
      all_sents.append(el)

  for i in all_sents:
    first_sent.append(i)
    if i[0] == '.':
      ind = all_sents.index(i)
      break
      
  second_sent = all_sents[ind::]
  del first_sent[-1]

  if len(first_sent) >= 4:
    pass

  else:
    parts_of_sentences2 = ['nsubj','ROOT', 'expl']
    sent = []
    for el in first_sent:
      if el in parts_of_sentences2:
        sent.append(el)
        if len(sent) >= 2:
          pass
        else:
          'the comma is not needed'

  index1 = int(list(sample[8].keys())[0])
  index2 = int(sample[2]) + 1
  indexes = range(index1, index2)
  error_area = [sample[7][i] for i in indexes]

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))

  error_area = error_area[0:-1]
  pprint('CORRECTION: ' + ' '.join(error_area) + ',')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

  print('\n')

In [348]:
#for el in dots_commas.values:
#  change_dots_commas(el)

In [349]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in dots_commas['corrected_text'].values)]

In [356]:
commas_colon = []
for el in punctuation.values:
  if str(el[4]).startswith(','):
    if str(el[5]).startswith(';'):
      commas_colon.append(el)


In [357]:
commas_colon = pandas.DataFrame(commas_colon)
commas_colon.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
commas_colon = commas_colon.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [364]:
def change_commas_colon(sample):
  tag = 'punctuation'
  parts_of_sentences = ['ROOT', 'relcl', 'nsubj', 'expl', 'conj', 'ccomp', 'aux', 
                      'neg', 'nsubjpass','punct','intj', 'acl']
  
  all_sents = []
  first_sent = []
  for el in sample[11].values():

    if el[-1] in parts_of_sentences:
      all_sents.append(el)

  for i in all_sents:
    first_sent.append(i)
    if i[0] == '.':
      ind = all_sents.index(i)
      break
      
  second_sent = all_sents[ind::]
  del first_sent[-1]

  if len(first_sent) >= 4:
    pass

  else:
    parts_of_sentences2 = ['nsubj','ROOT', 'expl']
    sent = []
    for el in first_sent:
      if el in parts_of_sentences2:
        sent.append(el)
        if len(sent) >= 2:
          pass
        else:
          'the comma is not needed'

  index1 = int(list(sample[8].keys())[0])
  index2 = int(sample[2]) + 2
  indexes = range(index1, index2)
  error_area = [sample[7][i] for i in indexes]

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))

  error_area = error_area[0:-1]
  pprint('CORRECTION: ' + ' '.join(error_area[0:-1]) + ';')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

  print('\n')

In [366]:
#for el in commas_colon.values:
 # change_commas_colon(el)

In [367]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in commas_colon['corrected_text'].values)]

In [370]:
delete_commas2 = []
for el in punctuation.values:
  if str(el[4]).startswith(','):
    if str(el[5]).isalpha():
      delete_commas2.append(el)

In [371]:
delete_commas2 = pandas.DataFrame(delete_commas2)
delete_commas2.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
delete_commas2 = delete_commas2.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [407]:
def change_delete_commas2(sample):
  tag = 'punctuation'
  parts_of_sentences = ['ROOT', 'relcl', 'nsubj', 'expl', 'conj', 'ccomp', 'aux', 
                      'neg', 'nsubjpass','punct','intj', 'acl']
  
  all_sents = []
  first_sent = []
  for el in sample[11].values():
    if el[-1] in parts_of_sentences:
      all_sents.append(el)

  for i in all_sents:
    first_sent.append(i)
    if i[0] == '.':
      ind = all_sents.index(i)
      break
      
  second_sent = all_sents[ind::]
  del first_sent[-1]

  if len(first_sent) >= 4:
    pass

  else:
    parts_of_sentences2 = ['nsubj','ROOT', 'expl']
    sent = []
    for el in first_sent:
      if el in parts_of_sentences2:
        sent.append(el)
        if len(sent) >= 2:
          pass
        else:
          'the comma is not needed'

  index1 = int(list(sample[8].keys())[0])
  index2 = int(sample[2]) + 1
  indexes = range(index1, index2)
  error_area = [sample[7][i] for i in indexes]

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))

  error_area = error_area
  pprint('CORRECTION: ' + ' '.join(error_area[0:-1]))
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

  if len(list(set(list(sample[4].split()[1])) ^ set(list(sample[5])))) < 4:
    element = sample[4]
    f = re.findall(r'[a-zA-Z]+', element)
    a = re.findall(r'\d+', element)
    k = re.findall(r'\W', element)
    all = f+a+k
    if ' ' in all:
      all.remove(' ')
    length = len(all)
    index = int(sample[2]) + length -1

    tag = 'spelling'
    pprint('CORRECTION: ' + str(sample[5].split()[0]))
    pprint('ERROR AREA: '+  str(sample[7][index]))
    pprint('TAG: ' + tag)


In [409]:
#for el in delete_commas2.values:
#  change_delete_commas2(el)

In [405]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in delete_commas2['corrected_text'].values)]

In [456]:
conj_and_articles = []
for el in punctuation.values:
  try:
    if el[4].isalpha():
      element = el[5]
      f = re.findall(r'[a-zA-Z]+', element)
      a = re.findall(r'\d+', element)
      k = re.findall(r'\W', element)
      all = f+a+k
      if ' ' in all:
        all.remove(' ')
      length = len(all)
      index = int(el[2]) + length - 1
      if el[11][index][1] == 'DET':
        conj_and_articles.append(el)
      elif el[11][index][1] == 'CCONJ':
        conj_and_articles.append(el)
  except:
    pass


In [457]:
conj_and_articles = pandas.DataFrame(conj_and_articles)
conj_and_articles.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
conj_and_articles = conj_and_articles.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [478]:
def change_conj_and_articles(sample):
  tag = 'punctuation'
  parts_of_sentences = ['ROOT', 'relcl', 'nsubj', 'expl', 'conj', 'ccomp', 'aux', 
                      'neg', 'nsubjpass','punct','intj', 'acl']
  
  all_sents = []
  first_sent = []
  for el in sample[11].values():
    if el[-1] in parts_of_sentences:
      all_sents.append(el)

  for i in all_sents:
    first_sent.append(i)
    if i[0] == '.':
      ind = all_sents.index(i)
      break
      
  second_sent = all_sents[ind::]
  del first_sent[-1]

  if len(first_sent) >= 4:
    pass

  else:
    parts_of_sentences2 = ['nsubj','ROOT', 'expl']
    sent = []
    for el in first_sent:
      if el in parts_of_sentences2:
        sent.append(el)
        if len(sent) >= 2:
          pass
        else:
          'the comma is not needed'

  index1 = int(list(sample[8].keys())[0])
  index2 = int(sample[2]) + 1
  indexes = range(index1, index2)
  error_area = [sample[7][i] for i in indexes]

  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))

  error_area = error_area
  pprint('CORRECTION: ' + ' '.join(error_area) + ',')
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

  element = sample[5]
  f = re.findall(r'[a-zA-Z]+', element)
  a = re.findall(r'\d+', element)
  k = re.findall(r'\W', element)
  all = f+a+k
  if ' ' in all:
    all.remove(' ')
  length = len(all)
  index = int(sample[2]) + length -1
  if sample[11][index][0] == 'a':
    element = str(sample[4])


    f = re.findall(r'[a-zA-Z]+', element)
    a = re.findall(r'\d+', element)
    k = re.findall(r'\W', element)
    all = f+a+k
    if ' ' in all:
      all.remove(' ')
    length = len(all)
    
    tag = 'articles'
    index = int(sample[2]) + length 
    pprint('CORRECTION: ' + 'a ' + str(sample[7][index]))
    pprint('ERROR AREA: '+  str(sample[7][index]))
    pprint('TAG: ' + tag)

  elif sample[11][index][1] == 'CCONJ':
    tag = 'absence'
    index = sample[2]
    pprint('CORRECTION: ' + str(sample[5]))
    pprint('ERROR AREA: '+ str(sample[7][index]))
    pprint('TAG: ' + tag)





In [479]:
#for el in conj_and_articles.values:
#  change_conj_and_articles(el)

In [483]:
solo_punct = []
for el in punctuation.values:
  if str(el[4]) in string.punctuation:
    if str(el[5]) in string.punctuation:
      solo_punct.append(el)

In [484]:
solo_punct = pandas.DataFrame(solo_punct)
solo_punct.rename(columns = {0: 'text_id', 1 : 'sentence_id', 2 : 'token_id', 3 : 'mistake_type', 4 : 'error_span', 5 : 'correction', 6 : 'full_text', 7 : 'tokens_ids', 
                               8 : 'dependencies', 9 : 'corrected_text', 10 : 'corrected_with_ids', 11 : 'dependencies_heptabot'}, inplace = True)
solo_punct = solo_punct.drop_duplicates(subset=['corrected_text'], ignore_index=True)

In [490]:
def change_solo_punct(sample):
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  tag = 'punctuation'


  i = list(sample[7].keys())[0]
  index1 = list(sample[7].keys())[0]
  while i != el[2]:
    i += 1
    if el[7][i] == ',':
      index1 = i + 1

  index2 = int(el[2]) 
  indexes = range(index1, index2 + 1)

  error_area = []
  for b in indexes:
    error_area.append(str(el[7][b]))


  pprint('CORRECTION: ' + ' '.join(error_area[0:-1])+str(sample[5]))
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

In [492]:
#for el in solo_punct.values:
#  change_solo_punct(el)

In [512]:
def change_other_cases(sample):
  pprint("USER'S TEXT: " + sample[6])
  pprint('CORRECTED TEXT: ' + str(sample[9]))
  tag = 'punctuation'


  element = sample[4]
  f = re.findall(r'[a-zA-Z]+', element)
  a = re.findall(r'\d+', element)
  k = re.findall(r'\W', element)
  all = f+a+k
  if ' ' in all:
    all.remove(' ')
  length = len(all)
  index2 = int(sample[2]) + length
  index1 = int(sample[2])
  indexes = range(index1, index2)
  
  error_area = []
  for ind in indexes:
    error_area.append(str(sample[7][ind]))
  pprint('CORRECTION: ' + str(sample[5]))
  pprint('ERROR AREA: '+ ' '.join(error_area))
  pprint('TAG: ' + tag)

In [514]:
for el in punctuation.values:
  try:
    change_other_cases(el)
  except:
    print(el)

("USER'S TEXT: As Africa Today had recenly reported 78% of students living in "
 'Africa simply do not have enough income to pay for University. Moreover, '
 'there are some professions requiring special education and equipment.')
('CORRECTED TEXT: As Africa Today had recenly reported, 78% of students living '
 'in Africa simply do not have enough income to pay for University. Moreover, '
 'there are some professions requiring special education and equipment.')
'CORRECTION: , 78%'
'ERROR AREA: 78 %'
'TAG: punctuation'
("USER'S TEXT: Only germany saw a slight increase from 1990 to 1995 To sum up, "
 'there is a general tendency of a decline in income difference between men '
 'and women, which is quite positive.')
('CORRECTED TEXT: Only germany saw a slight increase from 1990 to 1995.0 To '
 'sum up, there is a general tendency of a decline in income difference '
 'between men and women, which is quite positive.')
[592.0 6.0 128.0 'punct' 1995.0 1995.0
 'Only germany saw a slight increa

In [494]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in solo_punct['corrected_text'].values)]

In [495]:
punctuation

,text_id,sentence_id,token_id,mistake_type,error_span,correction,full_text,tokens_ids,dependencies,corrected_text,corrected_with_ids,dependencies_heptabot
16,731.0,6.0,128.0,punct,78%,", 78%",As Africa Today had recenly reported 78% of st...,"{122: 'As', 123: 'Africa', 124: 'Today', 125: ...","{122: ['As', 'SCONJ', 'IN', 'mark'], 123: ['Af...","As Africa Today had recenly reported, 78% of s...","{122: 'As', 123: 'Africa', 124: 'Today', 125: ...","{122: ['As', 'SCONJ', 'IN', 'mark'], 123: ['Af..."
127,592.0,6.0,128.0,punct,1995.0,1995.0,Only germany saw a slight increase from 1990 t...,"{119: 'Only', 120: 'germany', 121: 'saw', 122:...","{119: ['Only', 'ADV', 'RB', 'advmod'], 120: ['...",Only germany saw a slight increase from 1990 t...,"{119: 'Only', 120: 'germany', 121: 'saw', 122:...","{119: ['Only', 'ADV', 'RB', 'advmod'], 120: ['..."
140,1041.0,5.0,82.0,punct,1995.0,1995.0,"As we can see Japans dinamics on the graph, sh...","{59: 'As', 60: 'we', 61: 'can', 62: 'see', 63:...","{59: ['As', 'SCONJ', 'IN', 'mark'], 60: ['we',...","As we can see Japans dinamics on the graph, sh...","{59: 'As', 60: 'we', 61: 'can', 62: 'see', 63:...","{59: ['As', 'SCONJ', 'IN', 'mark'], 60: ['we',..."
275,344.0,8.0,187.0,punct,",",. A,In UK difference decreased from around 36 by 1...,"{177: 'In', 178: 'UK', 179: 'difference', 180:...","{177: ['In', 'ADP', 'IN', 'prep'], 178: ['UK',...",In UK difference decreased from around 36 by 1...,"{177: 'In', 178: 'UK', 179: 'difference', 180:...","{177: ['In', 'ADP', 'IN', 'prep'], 178: ['UK',..."
921,803.0,6.0,112.0,punct,", it",. That,"Moreover, school eduction or at least elementa...","{98: 'Moreover', 99: ',', 100: 'school', 101: ...","{98: ['Moreover', 'ADV', 'RB', 'advmod'], 99: ...","Moreover, school eduction or at least elementa...","{98: 'Moreover', 99: ',', 100: 'school', 101: ...","{98: ['Moreover', 'ADV', 'RB', 'advmod'], 99: ..."
1083,370.0,4.0,65.0,punct,. Because,because,"37 On the one hand, there will be difficult fo...","{43: '37', 44: 'On', 45: 'the', 46: 'one', 47:...","{43: ['37', 'NUM', 'CD', 'ROOT'], 44: ['On', '...","37 On the one hand, there will be difficult fo...","{43: '37', 44: 'On', 45: 'the', 46: 'one', 47:...","{43: ['37', 'NUM', 'CD', 'ROOT'], 44: ['On', '..."
1096,323.0,14.0,247.0,punct,". Because,",", because","First, a free education have to be given those...","{230: 'First', 231: ',', 232: 'a', 233: 'free'...","{230: ['First', 'ADV', 'RB', 'advmod'], 231: [...","First, a free education have to be given those...","{230: 'First', 231: ',', 232: 'a', 233: 'free'...","{230: ['First', 'ADV', 'RB', 'advmod'], 231: [..."
1097,1598.0,10.0,285.0,punct,. But,",",Although there are some people who justify edu...,"{246: 'Although', 247: 'there', 248: 'are', 24...","{246: ['Although', 'SCONJ', 'IN', 'mark'], 247...",Although there are some people who justify edu...,"{246: 'Although', 247: 'there', 248: 'are', 24...","{246: ['Although', 'SCONJ', 'IN', 'mark'], 247..."
1100,852.0,5.0,137.0,punct,. Gradual,gradual,Japan is also a bit different from the other c...,"{121: 'Japan', 122: 'is', 123: 'also', 124: 'a...","{121: ['Japan', 'PROPN', 'NNP', 'nsubj'], 122:...",Japan is also a bit different from the other c...,"{121: 'Japan', 122: 'is', 123: 'also', 124: 'a...","{121: ['Japan', 'PROPN', 'NNP', 'nsubj'], 122:..."
1101,1509.0,16.0,275.0,punct,. i,", I",In conclusion. i want to say that in today's w...,"{273: 'In', 274: 'conclusion', 275: '.', 276: ...","{273: ['In', 'ADP', 'IN', 'ROOT'], 274: ['conc...","In conclusion, I want to say that in today's w...","{273: 'In', 274: 'conclusion', 275: ',', 276: ...","{273: ['In', 'ADP', 'IN', 'prep'], 274: ['conc..."


In [480]:
punctuation = punctuation[punctuation['corrected_text'].apply(lambda x: x not in conj_and_articles['corrected_text'].values)]

In [481]:
punctuation

,text_id,sentence_id,token_id,mistake_type,error_span,correction,full_text,tokens_ids,dependencies,corrected_text,corrected_with_ids,dependencies_heptabot
16,731.0,6.0,128.0,punct,78%,", 78%",As Africa Today had recenly reported 78% of st...,"{122: 'As', 123: 'Africa', 124: 'Today', 125: ...","{122: ['As', 'SCONJ', 'IN', 'mark'], 123: ['Af...","As Africa Today had recenly reported, 78% of s...","{122: 'As', 123: 'Africa', 124: 'Today', 125: ...","{122: ['As', 'SCONJ', 'IN', 'mark'], 123: ['Af..."
127,592.0,6.0,128.0,punct,1995.0,1995.0,Only germany saw a slight increase from 1990 t...,"{119: 'Only', 120: 'germany', 121: 'saw', 122:...","{119: ['Only', 'ADV', 'RB', 'advmod'], 120: ['...",Only germany saw a slight increase from 1990 t...,"{119: 'Only', 120: 'germany', 121: 'saw', 122:...","{119: ['Only', 'ADV', 'RB', 'advmod'], 120: ['..."
140,1041.0,5.0,82.0,punct,1995.0,1995.0,"As we can see Japans dinamics on the graph, sh...","{59: 'As', 60: 'we', 61: 'can', 62: 'see', 63:...","{59: ['As', 'SCONJ', 'IN', 'mark'], 60: ['we',...","As we can see Japans dinamics on the graph, sh...","{59: 'As', 60: 'we', 61: 'can', 62: 'see', 63:...","{59: ['As', 'SCONJ', 'IN', 'mark'], 60: ['we',..."
275,344.0,8.0,187.0,punct,",",. A,In UK difference decreased from around 36 by 1...,"{177: 'In', 178: 'UK', 179: 'difference', 180:...","{177: ['In', 'ADP', 'IN', 'prep'], 178: ['UK',...",In UK difference decreased from around 36 by 1...,"{177: 'In', 178: 'UK', 179: 'difference', 180:...","{177: ['In', 'ADP', 'IN', 'prep'], 178: ['UK',..."
921,803.0,6.0,112.0,punct,", it",. That,"Moreover, school eduction or at least elementa...","{98: 'Moreover', 99: ',', 100: 'school', 101: ...","{98: ['Moreover', 'ADV', 'RB', 'advmod'], 99: ...","Moreover, school eduction or at least elementa...","{98: 'Moreover', 99: ',', 100: 'school', 101: ...","{98: ['Moreover', 'ADV', 'RB', 'advmod'], 99: ..."
1043,6.0,8.0,221.0,punct,.,?,"For instance, a lot of famous people as Lomono...","{175: 'For', 176: 'instance', 177: ',', 178: '...","{175: ['For', 'ADP', 'IN', 'prep'], 176: ['ins...","For instance, a lot of famous people as Lomono...","{175: 'For', 176: 'instance', 177: ',', 178: '...","{175: ['For', 'ADP', 'IN', 'prep'], 176: ['ins..."
1044,15.0,8.0,131.0,punct,.,",",In Germany the difference plummed by more than...,"{118: 'In', 119: 'Germany', 120: 'the', 121: '...","{118: ['In', 'ADP', 'IN', 'prep'], 119: ['Germ...",In Germany the difference plummed by more than...,"{118: 'In', 119: 'Germany', 120: 'the', 121: '...","{118: ['In', 'ADP', 'IN', 'prep'], 119: ['Germ..."
1045,25.0,3.0,58.0,punct,.,",",According to the graph. in 1980 USA rate was a...,"{54: 'According', 55: 'to', 56: 'the', 57: 'gr...","{54: ['According', 'VERB', 'VBG', 'ROOT'], 55:...","According to the graph, in 1980 USA rate was a...","{54: 'According', 55: 'to', 56: 'the', 57: 'gr...","{54: ['According', 'VERB', 'VBG', 'prep'], 55:..."
1046,54.0,5.0,122.0,punct,.,?,"Thus, how can perspective programmer get into ...","{100: 'Thus', 101: ',', 102: 'how', 103: 'can'...","{100: ['Thus', 'ADV', 'RB', 'advmod'], 101: ['...","Thus, how can perspective programmer get into ...","{100: 'Thus', 101: ',', 102: 'how', 103: 'can'...","{100: ['Thus', 'ADV', 'RB', 'advmod'], 101: ['..."
1047,58.0,9.0,199.0,punct,.,?,"Of course, there were, are and will be people ...","{169: 'Of', 170: 'course', 171: ',', 172: 'the...","{169: ['Of', 'ADV', 'RB', 'advmod'], 170: ['co...","Of course, there were, are and will be people ...","{169: 'Of', 170: 'course', 171: ',', 172: 'the...","{169: ['Of', 'ADV', 'RB', 'advmod'], 170: ['co..."


In [402]:
for el in punctuation.values:
  print(el[4])
  print(el[5])
  print(el[6])
  print(el[9])
  print('\n')

78%
, 78%
As Africa Today had recenly reported 78% of students living in Africa simply do not have enough income to pay for University. Moreover, there are some professions requiring special education and equipment.
As Africa Today had recenly reported, 78% of students living in Africa simply do not have enough income to pay for University. Moreover, there are some professions requiring special education and equipment.


1995.0
1995.0
Only germany saw a slight increase from 1990 to 1995 To sum up, there is a general tendency of a decline in income difference between men and women, which is quite positive.
Only germany saw a slight increase from 1990 to 1995.0 To sum up, there is a general tendency of a decline in income difference between men and women, which is quite positive.


1995.0
1995.0
As we can see Japans dinamics on the graph, shows stable decrease of unequality of incomes for 10 years since 1985 to 1995 Those dinamic is represents all of the countries on the graph, exept the